In [2]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

c:\Users\srkap\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load dataset
file_path = "C:/Users/srkap/Downloads/virtual_assistant amruvahini final/virtual_assistant amruvahini final/dataset/avcoe.csv"
df = pd.read_csv(file_path)


In [4]:
# Load pre-trained BERT model and tokenizer
MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)

c:\Users\srkap\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\srkap\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [5]:
# Function to generate embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # Extract CLS token embedding


In [6]:
# Compute embeddings for all questions
df["Embeddings"] = df["Questions"].apply(lambda x: get_embedding(x))

In [7]:
# Function to find the most relevant answer
def get_answer(query):
    query_embedding = get_embedding(query)
    similarities = [cosine_similarity(query_embedding, emb.reshape(1, -1))[0][0] for emb in df["Embeddings"]]
    best_match_idx = np.argmax(similarities)
    return df.iloc[best_match_idx]["Answers"], similarities[best_match_idx]


In [8]:
# Example usage
query = "How can I apply for admission?"
answer, confidence = get_answer(query)
print(f"Query: {query}")
print(f"Answer: {answer}")
print(f"Confidence: {confidence:.4f}")

Query: How can I apply for admission?
Answer: You can apply for admission to AVCOE through their official website by filling out the online application form and submitting the required documents.
Confidence: 0.9843
